In [134]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf

In [135]:
df = pd.read_csv('dataset.csv')

In [136]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,df263d996281d984952c07998dc54358,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,f0ade77b43923b38237db569b016ba25,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,f0ade77b43923b38237db569b016ba25,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,f0ade77b43923b38237db569b016ba25,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,f0ade77b43923b38237db569b016ba25,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0


# Generic data about datasets

In [137]:
df.shape

(197428, 14)

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   market_id                 196441 non-null  float64
 1   created_at                197428 non-null  object 
 2   actual_delivery_time      197421 non-null  object 
 3   store_id                  197428 non-null  object 
 4   store_primary_category    192668 non-null  object 
 5   order_protocol            196433 non-null  float64
 6   total_items               197428 non-null  int64  
 7   subtotal                  197428 non-null  int64  
 8   num_distinct_items        197428 non-null  int64  
 9   min_item_price            197428 non-null  int64  
 10  max_item_price            197428 non-null  int64  
 11  total_onshift_partners    181166 non-null  float64
 12  total_busy_partners       181166 non-null  float64
 13  total_outstanding_orders  181166 non-null  f

# NULL Values check

In [139]:
df.isna().sum()

market_id                     987
created_at                      0
actual_delivery_time            7
store_id                        0
store_primary_category       4760
order_protocol                995
total_items                     0
subtotal                        0
num_distinct_items              0
min_item_price                  0
max_item_price                  0
total_onshift_partners      16262
total_busy_partners         16262
total_outstanding_orders    16262
dtype: int64

# EDA

In [140]:
df.loc[df['market_id'].isna()]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
45,NaN,2015-02-09 03:27:37,2015-02-09 04:22:18,ea119a40c1592979f51819b0bd38d39d,italian,1.0,2,2400,2,600,1800,94.0,92.0,149.0
182,NaN,2015-02-01 05:32:34,2015-02-01 06:01:21,a87ff679a2f3e71d9181a67b7542122c,mediterranean,3.0,2,1845,2,600,1245,33.0,23.0,30.0
970,NaN,2015-02-17 02:17:43,2015-02-17 03:15:14,fe8c15fed5f808006ce95eddb7366e35,NaN,NaN,3,3400,3,1000,1200,36.0,36.0,52.0
1126,NaN,2015-02-18 03:50:52,2015-02-18 04:15:09,aa2a77371374094fe9e0bc1de3f94ed9,sandwich,2.0,1,1395,1,795,795,89.0,71.0,72.0
1625,NaN,2015-02-17 03:49:46,2015-02-17 04:21:27,86311dbe35f1b6c5166365165602f54d,pizza,NaN,1,1699,1,1399,1399,44.0,40.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196027,NaN,2015-01-24 02:59:19,2015-01-24 04:16:30,84d9ee44e457ddef7f2c4f25dc8fa865,NaN,NaN,2,1998,2,898,999,129.0,130.0,232.0
196561,NaN,2015-02-02 20:49:57,2015-02-02 21:26:34,07042ac7d03d3b9911a00da43ce0079a,NaN,NaN,4,1975,3,250,750,33.0,34.0,40.0
197170,NaN,2015-02-01 01:29:54,2015-02-01 01:50:18,17e62166fc8586dfa4d1bc0e1742c08b,vietnamese,5.0,4,2495,4,350,795,74.0,58.0,64.0
197171,NaN,2015-02-16 19:16:44,2015-02-16 19:56:03,17e62166fc8586dfa4d1bc0e1742c08b,vietnamese,5.0,4,2830,2,195,795,50.0,49.0,56.0


In [141]:
df[['market_id','store_id']]

,market_id,store_id
0,1.0,df263d996281d984952c07998dc54358
1,2.0,f0ade77b43923b38237db569b016ba25
2,3.0,f0ade77b43923b38237db569b016ba25
3,3.0,f0ade77b43923b38237db569b016ba25
4,3.0,f0ade77b43923b38237db569b016ba25
...,...,...
197423,1.0,a914ecef9c12ffdb9bede64bb703d877
197424,1.0,a914ecef9c12ffdb9bede64bb703d877
197425,1.0,a914ecef9c12ffdb9bede64bb703d877
197426,1.0,c81e155d85dae5430a8cee6f2242e82c


In [142]:
# df.loc[df['store_id']=='a914ecef9c12ffdb9bede64bb703d877'][['market_id','store_id']].values[0][0],df.loc[df['store_id']=='a914ecef9c12ffdb9bede64bb703d877'][['market_id','store_id']].values[0][1]

In [143]:
store_id_market_id_mapping = df[['store_id','market_id']]
store_id_market_id_mapping = store_id_market_id_mapping.dropna()
store_id_market_id_mapping

,store_id,market_id
0,df263d996281d984952c07998dc54358,1.0
1,f0ade77b43923b38237db569b016ba25,2.0
2,f0ade77b43923b38237db569b016ba25,3.0
3,f0ade77b43923b38237db569b016ba25,3.0
4,f0ade77b43923b38237db569b016ba25,3.0
...,...,...
197423,a914ecef9c12ffdb9bede64bb703d877,1.0
197424,a914ecef9c12ffdb9bede64bb703d877,1.0
197425,a914ecef9c12ffdb9bede64bb703d877,1.0
197426,c81e155d85dae5430a8cee6f2242e82c,1.0


In [144]:
store_id_market_id_mapping

,store_id,market_id
0,df263d996281d984952c07998dc54358,1.0
1,f0ade77b43923b38237db569b016ba25,2.0
2,f0ade77b43923b38237db569b016ba25,3.0
3,f0ade77b43923b38237db569b016ba25,3.0
4,f0ade77b43923b38237db569b016ba25,3.0
...,...,...
197423,a914ecef9c12ffdb9bede64bb703d877,1.0
197424,a914ecef9c12ffdb9bede64bb703d877,1.0
197425,a914ecef9c12ffdb9bede64bb703d877,1.0
197426,c81e155d85dae5430a8cee6f2242e82c,1.0


In [145]:
# tmp = store_id_market_id_mapping.loc[store_id_market_id_mapping['store_id']=='f0ade77b43923b38237db569b016ba25']
 
# tmp.groupby('store_id')['market_id'].apply(lambda x : x.mode().iloc[0]).iloc[0]


In [146]:
from tqdm import tqdm

In [147]:
# def get_store_and_market_id_mapping(store_ids):
#     store_id_and_market_id_mapping = {}
#     for store_id in tqdm(store_ids):
#         tmp_store_id = store_id_market_id_mapping.loc[store_id_market_id_mapping['store_id']==store_id]
#         max_value_market_id = tmp_store_id.groupby('store_id')['market_id'].apply(lambda x : x.mode().iloc[0]).iloc[0]
#         if(store_id not in store_id_and_market_id_mapping.keys()):
#             store_id_and_market_id_mapping[store_id] = max_value_market_id
#     return store_id_and_market_id_mapping

In [148]:
# get_store_and_market_id_mapping = get_store_and_market_id_mapping(set(store_id_market_id_mapping['store_id']))

In [149]:
# list(get_store_and_market_id_mapping.keys())

In [150]:
df_store_id_and_market_id_mapping = pd.read_csv('get_store_and_market_id_mapping.csv')

In [151]:
df_store_id_and_market_id_mapping

,store_id,market_id
0,8e200fc779d0a8e7eaba42e877f0a5c0,5.0
1,1b9e43c170cd3fc59624a18663b8d4d2,2.0
2,e0d2fe50debfaec6b2d7bafdd9d936c8,2.0
3,84f5ddd735176becc72c3b1ff424149e,6.0
4,e57edfc7529f0c7b21788231308caeab,3.0
...,...,...
6735,0e4e946668cf2afc4299b462b812caca,1.0
6736,939b9fed93c76ce9339b8aa1b2d5c57c,6.0
6737,1690bccd010b308cd33989d3819ed96a,3.0
6738,57cd30d9088b0185cf0ebca1a472ff1d,1.0


In [152]:
df['market_id'][197171]

nan

In [153]:
df_store_id_and_market_id_mapping.loc[df_store_id_and_market_id_mapping['store_id']=='ea119a40c1592979f51819b0bd38d39d']['market_id'].values[0]

4.0

# Now Fixing the Missing Market_id

In [154]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,df263d996281d984952c07998dc54358,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,f0ade77b43923b38237db569b016ba25,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,f0ade77b43923b38237db569b016ba25,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,f0ade77b43923b38237db569b016ba25,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,f0ade77b43923b38237db569b016ba25,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0


In [155]:
df.loc[df['market_id'].isna()]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
45,NaN,2015-02-09 03:27:37,2015-02-09 04:22:18,ea119a40c1592979f51819b0bd38d39d,italian,1.0,2,2400,2,600,1800,94.0,92.0,149.0
182,NaN,2015-02-01 05:32:34,2015-02-01 06:01:21,a87ff679a2f3e71d9181a67b7542122c,mediterranean,3.0,2,1845,2,600,1245,33.0,23.0,30.0
970,NaN,2015-02-17 02:17:43,2015-02-17 03:15:14,fe8c15fed5f808006ce95eddb7366e35,NaN,NaN,3,3400,3,1000,1200,36.0,36.0,52.0
1126,NaN,2015-02-18 03:50:52,2015-02-18 04:15:09,aa2a77371374094fe9e0bc1de3f94ed9,sandwich,2.0,1,1395,1,795,795,89.0,71.0,72.0
1625,NaN,2015-02-17 03:49:46,2015-02-17 04:21:27,86311dbe35f1b6c5166365165602f54d,pizza,NaN,1,1699,1,1399,1399,44.0,40.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196027,NaN,2015-01-24 02:59:19,2015-01-24 04:16:30,84d9ee44e457ddef7f2c4f25dc8fa865,NaN,NaN,2,1998,2,898,999,129.0,130.0,232.0
196561,NaN,2015-02-02 20:49:57,2015-02-02 21:26:34,07042ac7d03d3b9911a00da43ce0079a,NaN,NaN,4,1975,3,250,750,33.0,34.0,40.0
197170,NaN,2015-02-01 01:29:54,2015-02-01 01:50:18,17e62166fc8586dfa4d1bc0e1742c08b,vietnamese,5.0,4,2495,4,350,795,74.0,58.0,64.0
197171,NaN,2015-02-16 19:16:44,2015-02-16 19:56:03,17e62166fc8586dfa4d1bc0e1742c08b,vietnamese,5.0,4,2830,2,195,795,50.0,49.0,56.0


In [156]:
def get_new_market_id(market_id,store_id):
    """
    This Function is helping to get fix market_id
    """
    new_market_id = []
    for market_id,store_id in zip(market_id,store_id):
        if(pd.isna(market_id)==True):
            if(len(df_store_id_and_market_id_mapping.loc[df_store_id_and_market_id_mapping['store_id']==store_id]['market_id'].values)!=0):
                new_market_id.append(df_store_id_and_market_id_mapping.loc[df_store_id_and_market_id_mapping['store_id']==store_id]['market_id'].values[0])
            else:
                new_market_id.append(np.nan)
        else:
            new_market_id.append(market_id)
    return new_market_id


new_market_id = get_new_market_id(df['market_id'],df['store_id']) 

print(len(new_market_id))
        


197428


In [157]:
df['new_market_id'] = new_market_id

In [158]:
df['market_id'] = new_market_id

In [159]:
df.drop('new_market_id',axis=1,inplace=True)

# We have fixed market_id upto some extent

In [160]:
df.isna().sum()

market_id                       3
created_at                      0
actual_delivery_time            7
store_id                        0
store_primary_category       4760
order_protocol                995
total_items                     0
subtotal                        0
num_distinct_items              0
min_item_price                  0
max_item_price                  0
total_onshift_partners      16262
total_busy_partners         16262
total_outstanding_orders    16262
dtype: int64

In [161]:
from sklearn.impute import SimpleImputer

In [162]:
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')


In [163]:
df['market_id'] = imputer.fit_transform(pd.DataFrame(df['market_id']))

In [164]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,df263d996281d984952c07998dc54358,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,f0ade77b43923b38237db569b016ba25,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,f0ade77b43923b38237db569b016ba25,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,f0ade77b43923b38237db569b016ba25,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,f0ade77b43923b38237db569b016ba25,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0


In [165]:
df.isna().sum()

market_id                       0
created_at                      0
actual_delivery_time            7
store_id                        0
store_primary_category       4760
order_protocol                995
total_items                     0
subtotal                        0
num_distinct_items              0
min_item_price                  0
max_item_price                  0
total_onshift_partners      16262
total_busy_partners         16262
total_outstanding_orders    16262
dtype: int64

In [166]:
df.loc[df['actual_delivery_time'].isna()]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
109,3.0,2015-02-10 21:51:54,NaN,da353f402faf6cf475d4abd1450b0882,sandwich,3.0,1,1125,1,975,975,7.0,5.0,4.0
7670,2.0,2015-02-08 02:54:42,NaN,140f6969d5213fd0ece03148e62e461e,japanese,2.0,4,2860,3,390,690,131.0,123.0,197.0
78511,4.0,2015-02-15 02:15:45,NaN,1f0e3dad99908345f7439f8ffabdffc4,catering,1.0,9,5050,6,375,1125,91.0,75.0,167.0
115982,4.0,2015-02-16 01:52:49,NaN,b92894e4589f652dc3116cb3a8c48c08,pizza,3.0,2,2094,2,599,1195,53.0,53.0,102.0
140635,2.0,2015-02-15 02:21:42,NaN,0d73a25092e5c1c9769a9f3255caa65a,dessert,1.0,3,4210,3,865,1850,123.0,91.0,176.0
158967,2.0,2015-02-01 01:21:29,NaN,faacbcd5bf1d018912c116bf2783e9a1,mexican,5.0,5,3447,3,225,1349,90.0,88.0,109.0
170416,5.0,2015-02-01 01:36:33,NaN,6c7a107981f9c2a0ed55efce297bd1e2,fast,4.0,3,982,3,165,575,41.0,31.0,31.0


# Fixing actual delivery time NAN Values

In [168]:
df['actual_delivery_time'] = df['actual_delivery_time'].ffill(axis=0)

In [169]:
df.isna().sum()

market_id                       0
created_at                      0
actual_delivery_time            0
store_id                        0
store_primary_category       4760
order_protocol                995
total_items                     0
subtotal                        0
num_distinct_items              0
min_item_price                  0
max_item_price                  0
total_onshift_partners      16262
total_busy_partners         16262
total_outstanding_orders    16262
dtype: int64

In [176]:
df.loc[df['store_primary_category'].isna()]

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,f0ade77b43923b38237db569b016ba25,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,f0ade77b43923b38237db569b016ba25,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,f0ade77b43923b38237db569b016ba25,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0
5,3.0,2015-01-28 20:30:38,2015-01-28 21:08:58,f0ade77b43923b38237db569b016ba25,NaN,1.0,3,5000,3,1500,1900,2.0,2.0,2.0
6,3.0,2015-01-31 02:16:36,2015-01-31 02:43:00,f0ade77b43923b38237db569b016ba25,NaN,1.0,2,3900,2,1200,2700,10.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197210,1.0,2015-02-15 02:17:39,2015-02-15 03:21:16,77c493ec14246d748db3ee8fce0092db,NaN,1.0,3,2800,3,700,1400,24.0,22.0,25.0
197211,1.0,2015-02-03 01:58:27,2015-02-03 02:32:07,77c493ec14246d748db3ee8fce0092db,NaN,1.0,2,1400,2,700,700,22.0,22.0,24.0
197212,1.0,2015-02-12 02:42:13,2015-02-12 03:39:29,77c493ec14246d748db3ee8fce0092db,NaN,1.0,5,2800,5,300,700,28.0,28.0,21.0
197259,5.0,2015-02-10 01:32:37,2015-02-10 02:02:09,959776b99b006e5785c3a3364949ce47,NaN,NaN,1,849,1,849,849,25.0,13.0,20.0


In [188]:
df.loc[df['store_id']=='f0ade77b43923b38237db569b016ba25']

tmp_store_id = df.loc[df['store_id']=='f0ade77b43923b38237db569b016ba25']
max_value_market_id = tmp_store_id.groupby('store_id')['store_primary_category'].apply(lambda x : x.mode().iloc[0]).iloc[0]
print(max_value_market_id)

indian


In [189]:
# def get_store_and_store_primary_category_mapping(store_ids):
#     """
#     This Function will get store_id with store_primary_category value mapping
#     """
#     store_and_store_primary_category_mapping = {}
#     for store_id in tqdm(store_ids):
#         tmp_store_id = df.loc[df['store_id']==store_id]
#         max_value_store_primary_id = tmp_store_id.groupby('store_id')['store_primary_category'].apply(lambda x : x.mode().iloc[0]).iloc[0]
#         if(store_id not in store_and_store_primary_category_mapping.keys()):
#             store_and_store_primary_category_mapping[store_id] = max_value_store_primary_id
#     return store_and_store_primary_category_mapping

In [190]:
store_and_store_primary_category_mapping  = get_store_and_store_primary_category_mapping(set(df['store_id']))
len(store_and_store_primary_category_mapping)

  0%|          | 19/6743 [00:00<02:44, 40.84it/s]


IndexError: single positional indexer is out-of-bounds